# San Francisco Housing Cost Analysis

In this assignment, you will perform fundamental analysis for the San Francisco housing market to allow potential real estate investors to choose rental investment properties.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

## Load Data

In [ ]:
# Read the census data into a Pandas DataFrame
file_path = Path("../../week6_pyviz/unit_6_pythonic_monopoly/sfo_neighborhoods_census_data.csv")
#week6_pyviz/unit_6_pythonic_monopoly/sfo_neighborhoods_census_data.csv
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function.

**Hint:** Use the Pandas `groupby` function.

**Optional challenge:** Use the min, max, and std to scale the y limits of the chart.



In [ ]:
# Calculate the mean number of housing units per year (hint: use groupby) 
sfo_data_mean = sfo_data.groupby("year")['housing_units'].mean()
sfo_data_mean

In [ ]:
# Save the dataframe as a csv file
import pandas as pd
sfo_data_mean_df = sfo_data.groupby("year")['housing_units'].mean()
sfo_data_mean_df = pd.DataFrame(sfo_data_mean_df, columns= ["housing_units"])
sfo_data_mean_df.to_csv("../../week6_pyviz/unit_6_pythonic_monopoly/sfo_data_mean_df.csv")
sfo_data_mean_df

In [ ]:
# Use the Pandas plot function to plot the average housing units per year.
# Note: You will need to manually adjust the y limit of the chart using the min and max values from above.
fig_housing_units = plt.figure() 
plot_housing_units = sfo_data_mean.plot.bar().set_title("Housing Units in San Francisco from 2010 to 2016")
plt.show()

In [ ]:
# Optional Challenge: Use the min, max, and std to scale the y limits of the chart
max1 = sfo_data_mean.max()
min1 = sfo_data_mean.min()
std1 = sfo_data_mean.std()
sfo_data_mean.plot.bar(ylim=(min1-std1, max1+std1), title="Housing Units in San Francisco from 2010 to 2016")
plt.show()

- - - 

## Average Housing Costs in San Francisco Per Year

In this section, you will calculate the average monthly rent and the average price per square foot for each year. An investor may wish to better understand the sales price of the rental property over time. For example, a customer will want to know if they should expect an increase or decrease in the property value over time so they can determine how long to hold the rental property.  Plot the results as two line charts.

**Optional challenge:** Plot each line chart in a different color.

In [ ]:
# Calculate the average sale price per square foot and average gross rent
#sale_price_sqr_foot_mean = sale_price_sqr_foot.groupby("year")['sale_price_sqr_foot'].mean()
#gross_rent_mean = gross_rent.groupby("year")['gross_rent'].mean()
sfo_data_mean = sfo_data.groupby('year')['sale_price_sqr_foot', 'gross_rent'].mean()
sfo_data_mean = sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby('year').mean()
sfo_data_mean

In [ ]:
# Create two line charts, one to plot the average sale price per square foot and another for average montly rent

# Line chart for average sale price per square foot
fig_housing_units = plt.figure()
sfo_data_mean = sfo_data.groupby('year')['sale_price_sqr_foot'].mean().plot(color= "green", title="Average Price per SqFt by Year")

plt.show()


# Line chart for average montly rent
fig_housing_units = plt.figure()
sfo_data_mean = sfo_data[['gross_rent']].groupby('year').mean().plot(color = "red", title="Average Rent per Year")

plt.show()


- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create two interactive visulizations of average prices with a dropdown selector for the neighborhood. The first visualization will be a line plot showing the trend of average price per square foot over time for each neighborhood.  The second will be a line plot showing the trend of average montly rent over time for each neighborhood.

**Hint:** It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [ ]:
# Group by year and neighborhood and then create a new dataframe of the mean values
average_price_by_neighborhood = sfo_data.groupby(["year", "neighborhood"]).mean()
average_price_by_neighborhood.reset_index(inplace=True)
average_price_by_neighborhood.head()

In [ ]:
# Use hvplot to create an interactive line chart of the average price per sq ft.
# The plot should have a dropdown selector for the neighborhood
average_price_by_neighborhood.hvplot.line ("year", "sale_price_sqr_foot", groupby="neighborhood")


In [ ]:
# Use hvplot to create an interactive line chart of the average price per sq ft.
# The plot should have a dropdown selector for the neighborhood
average_price_by_neighborhood.hvplot.line ("year", "gross_rent", groupby = "neighborhood")

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price per square foot for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [ ]:
# Getting the data from the top 10 expensive neighborhoods to own
neighborhood_prices = sfo_data.groupby(['neighborhood']).mean()
top_ten_expensive = neighborhood_prices.sort_values('sale_price_sqr_foot', ascending = False).head(10).reset_index()
top_ten_expensive

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
top_ten_expensive.plot.bar(x="neighborhood", y="sale_price_sqr_foot", title="Top 10 Expensive Neighborhoods")

- - - 

## Comparing cost to purchase versus rental income

In this section, you will use `hvplot` to create an interactive visualization with a dropdown selector for the neighborhood. This visualization will feature a side-by-side comparison of average price per square foot versus average montly rent by year.  

**Hint:** Use the `hvplot` parameter, `groupby`, to create a dropdown selector for the neighborhood.

In [ ]:
# Fetch the previously generated DataFrame that was grouped by year and neighborhood
sfo_data_mean = sfo_data.groupby('year')['sale_price_sqr_foot', 'gross_rent']
sfo_data_mean = sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby('year')

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
#notFinal
top_ten_expensive.plot.bar(x="neighborhood", y="sale_price_sqr_foot", title="Top 10 Expensive Neighborhoods")

- - - 

## Neighborhood Map

In this section, you will read in neighborhoods location data and build an interactive map with the average house value per neighborhood. Use a `scatter_mapbox` from Plotly express to create the visualization. Remember, you will need your Mapbox API key for this.

### Load Location Data

In [ ]:
# Load neighborhoods coordinates data
nh_coordinates_path = Path("../../week6_pyviz/unit_6_pythonic_monopoly/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(nh_coordinates_path)
df_neighborhood_locations.head()

### Data Preparation

You will need to join the location data with the mean values per neighborhood.

1. Calculate the mean values for each neighborhood.

2. Join the average values with the neighborhood locations.

In [ ]:
# Calculate the mean values for each neighborhood
neighborhood_prices = neighborhood_prices.reset_index()
neighborhood_prices = neighborhood_prices.rename(columns = {"neighborhood": "Neighborhood"}) 
neighborhood_jt_df = pd.merge (neighborhood_prices,df_neighborhood_locations, on = "Neighborhood")
neighborhood_jt_df

In [ ]:
# Join the average values with the neighborhood locations

### Mapbox Visualization

Plot the average values per neighborhood using a Plotly express `scatter_mapbox` visualization.

In [ ]:
# Set the mapbox access token
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)
print(type(map_box_api))

# Create a scatter mapbox to analyze neighborhood info
scatter = px.scatter_mapbox(neighborhood_jt_df, lat ="Lat", lon ="Lon", color = "gross_rent", size = "sale_price_sqr_foot",zoom=11, size_max = 16)
scatter.show()

- - -

## Cost Analysis - Optional Challenge

In this section, you will use Plotly express to create visualizations that investors can use to interactively filter and explore various factors related to the house value of the San Francisco's neighborhoods. 

### Create a DataFrame showing the most expensive neighborhoods in San Francisco by year

In [ ]:
# Fetch the data from all expensive neighborhoods per year.

#top_ten_expensive = neighborhood_prices.sort_values('sale_price_sqr_foot', ascending = False, ).head(10).reset_index()
#top_ten_expensive groupby, "year"

#sfo_data_exp_year = sfo_data.groupby('year')['sale_price_sqr_foot', 'gross_rent', ascending = False]
#sfo_data_exp_year = sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby('year')
#sfo_data_exp_year
#sfo_data_mean = sfo_data_mean
#sfo_data_mean

#neighborhood_jt_per_year_df = pd.merge (top_ten_expensive, sfo_data_mean, on = "year")
#neighborhood_jt_per_year_df


### Create a parallel coordinates plot and parallel categories plot of most expensive neighborhoods in San Francisco per year


In [ ]:
# Parallel Categories Plot
# YOUR CODE HERE!

In [ ]:
# Parallel Coordinates Plot
# YOUR CODE HERE!

### Create a sunburst chart to conduct a costs analysis of most expensive neighborhoods in San Francisco per year

In [ ]:
# Sunburst Plot
## df=px.data.gapminder().query("year ==")
    #fig =px.sunburst(df, path=["",""]), values="pop",
               # color ="green", hover_data=["iso_alpha"])
    #fig.show()           